# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**、表示は仕様どおりの列名と順序

## 1) 問題

* `各ノードを Root / Inner / Leaf のいずれかに分類して返す。`
* 入力テーブル例: **Tree(id, p_id)**
* 出力仕様: **列** `id, type`
  **規則**

  * `p_id IS NULL` → `"Root"`
  * 親はあるが子が **存在しない** → `"Leaf"`
  * 親があり、かつ子が **存在する** → `"Inner"`
  * 結果順は任意

## 2) 最適解（単一クエリ）

> 子の有無を一度だけ集計し（CTE）、本体に左結合して `CASE` で型判定。

```sql
WITH child_count AS (
  SELECT
    p_id AS id,
    COUNT(*) AS cnt
  FROM Tree
  WHERE p_id IS NOT NULL
  GROUP BY p_id
)
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN cc.cnt IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN child_count AS cc
  ON cc.id = t.id;

Runtime 481 ms
Beats 31.78%

```

## 3) 代替解

> 相関 `EXISTS` を使い、子の存在だけを直接判定（ウィンドウや集計が重い場合に有効）。

```sql
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN EXISTS (SELECT 1 FROM Tree AS ch WHERE ch.p_id = t.id) THEN 'Inner'
    ELSE 'Leaf'
  END AS type
FROM Tree AS t;

Runtime 485 ms
Beats 29.67%
```

## 4) 要点解説

* **方針**: 「子の有無」を `GROUP BY p_id` で一度だけ集計 → 元テーブルに `LEFT JOIN` → `CASE`.
* **NULL / 重複**:

  * ルート判定は `p_id IS NULL` を明示。
  * `NOT IN` は使わず、`LEFT JOIN ... IS NULL` または `EXISTS` で安全に表現。
* **安定性**: 任意順なので `ORDER BY` 不要。大規模でも結合キー `Tree.id` / `Tree.p_id` に索引があれば高速。

## 5) 計算量（概算）

* `child_count` の集計: **O(N)**（ハッシュ集約）
* `LEFT JOIN` 結合: **O(N)** 近似（`id` / `p_id` にインデックス前提）
* 全体: **O(N)** ～ **O(N log N)**

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Tree id p_id] --> B[子数集計 GROUP BY p_id]
  B --> C[child_count id cnt]
  A --> D[LEFT JOIN child_count]
  D --> E[CASE で Root Inner Leaf 判定]
  E --> F[出力 id type]
```

パフォーマンス改善の余地としては、**集計（COUNT）をやめて “子を持つIDの集合” を `DISTINCT` で作る**形に置き換えるのが手堅いです。多くのワークロードで `GROUP BY + COUNT(*)` より **`SELECT DISTINCT` の方が軽く**、結合列も 1 列だけになるのでメモリも少なく済みます。

## 改善版（推奨）

```sql
WITH has_child AS (
  SELECT DISTINCT p_id AS id
  FROM Tree
  WHERE p_id IS NOT NULL
)
SELECT
  t.id,
  CASE
    WHEN t.p_id IS NULL THEN 'Root'
    WHEN hc.id IS NULL THEN 'Leaf'
    ELSE 'Inner'
  END AS type
FROM Tree AS t
LEFT JOIN has_child AS hc
  ON hc.id = t.id;

Runtime 430 ms
Beats 72.61%

```

**ポイント**

* `COUNT(*)` を落として `SELECT DISTINCT` に変更 → 集計コストと中間行サイズを削減
* 判定は `LEFT JOIN ... IS NULL` のままなのでロジックは不変
* スキャン回数は従来と同等だが、**中間結果の計算・転送量が減る**分だけ有利になりやすい

## 代替（条件分岐を 3 本に割る）

環境によっては、**セミジョイン最適化が効く `EXISTS / NOT EXISTS` の 3 本 `UNION ALL`** が速いことがあります。`p_id` にインデックスがあると特に効きやすいです。

```sql
SELECT id, 'Root' AS type
FROM Tree
WHERE p_id IS NULL

UNION ALL
SELECT t.id, 'Inner'
FROM Tree AS t
WHERE t.p_id IS NOT NULL
  AND EXISTS (SELECT 1 FROM Tree AS ch WHERE ch.p_id = t.id)

UNION ALL
SELECT t.id, 'Leaf'
FROM Tree AS t
WHERE t.p_id IS NOT NULL
  AND NOT EXISTS (SELECT 1 FROM Tree AS ch WHERE ch.p_id = t.id);

Runtime 439 ms
Beats 64.56%

```

**ポイント**

* 条件が絞られるため、`EXISTS` 側が **最初のヒットで即打ち切り**できる
* Optimizer のセミジョイン変換が効くと、**インデックス `Tree(p_id)` だけで高速**に走る

## インデックス（最重要）

クエリを書き換えるより効くことが多いのがこれです。

```sql
ALTER TABLE Tree
  ADD PRIMARY KEY (id),
  ADD INDEX idx_tree_pid (p_id);
```

* `child_count` / `has_child` / `EXISTS` いずれも **`p_id` へのインデックスがある前提で最速化**されます
* 結合側は `id` 参照なので `PRIMARY KEY(id)` があると良い（`id` はユニークとの仕様）

## そのほかの微調整

* CTE は MySQL では状況によって **マテリアライズ**されます。もし実測で悪化する場合は、CTEをやめて**派生表**に置き換えると最適化（プッシュダウン）が働くことがあります。

  ```sql
  SELECT t.id,
         CASE
           WHEN t.p_id IS NULL THEN 'Root'
           WHEN hc.id IS NULL THEN 'Leaf'
           ELSE 'Inner'
         END AS type
  FROM Tree AS t
  LEFT JOIN (
    SELECT DISTINCT p_id AS id
    FROM Tree
    WHERE p_id IS NOT NULL
  ) AS hc
    ON hc.id = t.id;

Runtime 426 ms
Beats 76.57%

  ```

* 返却順は任意のため `ORDER BY` は付けない（既に OK）。

---

### まとめ

* まずは **`DISTINCT` 版**（上の改善版）を試す
* 併せて **`INDEX(p_id)`** を必ず用意
* さらに詰めるなら **`UNION ALL` + `EXISTS/NOT EXISTS`** も計測

この 3 点で、提示の 481ms / 485ms からの短縮が見込めます。実データ分布次第ですが、`p_id` に偏りがある木では特に効果が出やすいです。

